In [38]:
import re
import numpy as np
import pandas as pd
import pickle
import json
from nltk.tokenize import TreebankWordTokenizer
import collections

In [39]:
#Converting CSVs to Dataframe objects
transcripts = pd.read_csv('ted-talks/transcripts.csv')
talk_information = pd.read_csv('ted-talks/ted_main.csv')

How it looks like: 

In [27]:
transcripts

,transcript,url
0,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...
1,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...
2,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...
3,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...
4,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...
5,Thank you. I have to tell you I'm both challen...,https://www.ted.com/talks/tony_robbins_asks_wh...
6,"On September 10, the morning of my seventh bir...",https://www.ted.com/talks/julia_sweeney_on_let...
7,I'm going to present three projects in rapid f...,https://www.ted.com/talks/joshua_prince_ramus_...
8,It's wonderful to be back. I love this wonderf...,https://www.ted.com/talks/dan_dennett_s_respon...
9,"I'm often asked, ""What surprised you about the...",https://www.ted.com/talks/rick_warren_on_a_lif...


In [28]:
talk_information

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869
5,672,"Tony Robbins discusses the ""invisible forces"" ...",1305,TED2006,1138838400,36,Tony Robbins,Tony Robbins: Why we do what we do,1,1151440680,"[{'id': 7, 'name': 'Funny', 'count': 1102}, {'...","[{'id': 229, 'hero': 'https://pe.tedcdn.com/im...",Life coach; expert in leadership psychology,"['business', 'culture', 'entertainment', 'goal...",Why we do what we do,https://www.ted.com/talks/tony_robbins_asks_wh...,20685401
6,919,When two young Mormon missionaries knock on Ju...,992,TED2006,1140739200,31,Julia Sweeney,Julia Sweeney: Letting go of God,1,1152490260,"[{'id': 3, 'name': 'Courageous', 'count': 477}...","[{'id': 22, 'hero': 'https://pe.tedcdn.com/ima...","Actor, comedian, playwright","['Christianity', 'God', 'atheism', 'comedy', '...",Letting go of God,https://www.ted.com/talks/julia_sweeney_on_let...,3769987
7,46,Architect Joshua Prince-Ramus takes the audien...,1198,TED2006,1140652800,19,Joshua Prince-Ramus,Joshua Prince-Ramus: Behind the design of Seat...,1,1152490260,"[{'id': 9, 'name': 'Ingenious', 'count': 195},...","[{'id': 750, 'hero': 'https://pe.tedcdn.com/im...",Architect,"['architecture', 'collaboration', 'culture', '...",Behind the design of Seattle's library,https://www.ted.com/talks/joshua_prince_ramus_...,967741
8,852,Philosopher Dan Dennett calls for religion -- ...,1485,TED2006,1138838400,32,Dan Dennett,Dan Dennett: Let's teach religion -- all relig...,1,1153181460,"[{'id': 3, 'name': 'Courageous', 'count': 562}...","[{'id': 71, 'hero': 'https://pe.tedcdn.com/ima...","Philosopher, cognitive scientist","['God', 'TED Brain Trust', 'atheism', 'brain',...",Let's teach religion -- all religion -- in sch...,https://www.ted.com/talks/dan_dennett_s_respon...,2567958
9,900,"Pastor Rick Warren, author of ""The Purpose-Dri...",1262,T

Example of how to grab a column within this dataframe object:

In [29]:
(talk_information['description'][2])

'New York Times columnist David Pogue takes aim at technology’s worst interface-design offenders, and provides encouraging examples of products that get it right. To funny things up, he bursts into song.'

Example of how to access an individual transcript using a Pandas dataframe:

In [30]:
print(transcripts['transcript'][0])

Good morning. How are you?(Laughter)It's been great, hasn't it? I've been blown away by the whole thing. In fact, I'm leaving.(Laughter)There have been three themes running through the conference which are relevant to what I want to talk about. One is the extraordinary evidence of human creativity in all of the presentations that we've had and in all of the people here. Just the variety of it and the range of it. The second is that it's put us in a place where we have no idea what's going to happen, in terms of the future. No idea how this may play out.I have an interest in education. Actually, what I find is everybody has an interest in education. Don't you? I find this very interesting. If you're at a dinner party, and you say you work in education — Actually, you're not often at dinner parties, frankly.(Laughter)If you work in education, you're not asked.(Laughter)And you're never asked back, curiously. That's strange to me. But if you are, and you say to somebody, you know, they sa

In [45]:
#creating good types

#tokenize method
def tokenize(text):
    """Returns a list of words that make up the text.
    
    Note: for simplicity, lowercase everything.
    Requirement: Use Regex to satisfy this function
    
    Params: {text: String}
    Returns: List
    """
    words = re.findall(r'[A-Za-z]+', text.lower())
    return words

assert((tokenize((talk_information['description'][0])))==['sir', 'ken', 'robinson', 'makes', 'an', 'entertaining', 
                                                          'and', 'profoundly', 'moving', 'case', 'for', 'creating', 
                                                          'an', 'education', 'system', 'that', 'nurtures', 
                                                          'rather', 'than', 'undermines', 'creativity'])

def tokenize_transcript(tokenize_method,input_transcript):
    """Returns a list of words contained in an entire transcript.
    Params: {tokenize_method: Function (a -> b),
             input_transcript: Tuple}
    Returns: List
    """
    final_lst = []
    for i in (range(0,len(input_transcript))):
        #print(tokenize_method(input_transcript[i]))
        final_lst = final_lst + tokenize_method(input_transcript[i])
    return final_lst

print(len(transcripts['transcript']))
all_words_total = tokenize_transcript(tokenize,talk_information['description'])

description_word_dict = (collections.Counter(all_words_total))
good_types_descriptions = {k:v for (k,v) in description_word_dict.items() if (v != 1)}

all_words_total_transcripts = tokenize_transcript(tokenize, transcripts['transcript'])
transcript_word_dict = (collections.Counter(all_words_total_transcripts))
good_types_transcripts = {k:v for (k,v) in transcript_word_dict.items() if (v!=1)}

print((good_types_transcripts))
#look at the cosine similarity code and see if you need anything else

2467
{'tedster': 21, 'caltrans': 3, 'postgraduate': 4, 'disrepair': 2, 'lettuces': 10, 'greenways': 3, 'isolates': 3, 'kapital': 2, 'uttar': 9, 'nondisabled': 2, 'slinky': 5, 'eyebrows': 14, 'bill': 307, 'fuller': 16, 'pinpoint': 27, 'vineet': 5, 'somi': 3, 'primarily': 97, 'district': 105, 'uncertainty': 111, 'alaska': 44, 'young': 1618, 'half': 1718, 'frazier': 2, 'go': 8090, 'associate': 57, 'raja': 2, 'attendance': 26, 'grumbling': 3, 'cloudbook': 2, 'obliquely': 2, 'shirky': 4, 'thunderous': 3, 'myod': 4, 'earrings': 7, 'softer': 8, 'distillery': 2, 'alarm': 75, 'procedure': 78, 'starves': 2, 'palling': 2, 'grave': 71, 'mesmerized': 7, 'spasms': 2, 'proved': 80, 'glamorized': 4, 'wiggle': 13, 'slipstream': 2, 'anaheim': 5, 'compute': 44, 'sundry': 3, 'integrators': 4, 'vivid': 40, 'poise': 2, 'episodes': 28, 'contextualized': 2, 'summary': 35, 'inserted': 20, 'circumvent': 5, 'venti': 2, 'eatery': 2, 'wipes': 6, 'cable': 78, 'tangent': 3, 'armies': 27, 'priest': 34, 'deflect': 11,

In [37]:
#MAYBE WE DO NOT NEED THIS, BUT DO NOT DELETE
#going to create pickle file next for good_types
#create a word frequency count across all words in all the transcripts

all_word_counts = dict()
with open("ted_main.json", "r") as f:
    transcripts_info = json.load(f)
#print((transcripts_info[0]['speaker_occupation']))
treebank_tokenizer = TreebankWordTokenizer()
flat_msgs = [m for transcript in transcripts_info for m in transcript]
print(flat_msgs)

['views', 'ratings', 'event', 'duration', 'main_speaker', 'published_date', 'comments', 'related_talks', 'title', 'film_date', 'tags', 'description', 'speaker_occupation', 'url', 'name', 'languages', 'num_speaker', 'views', 'ratings', 'event', 'duration', 'main_speaker', 'published_date', 'comments', 'related_talks', 'title', 'film_date', 'tags', 'description', 'speaker_occupation', 'url', 'name', 'languages', 'num_speaker', 'views', 'ratings', 'event', 'duration', 'main_speaker', 'published_date', 'comments', 'related_talks', 'title', 'film_date', 'tags', 'description', 'speaker_occupation', 'url', 'name', 'languages', 'num_speaker', 'views', 'ratings', 'event', 'duration', 'main_speaker', 'published_date', 'comments', 'related_talks', 'title', 'film_date', 'tags', 'description', 'speaker_occupation', 'url', 'name', 'languages', 'num_speaker', 'views', 'ratings', 'event', 'duration', 'main_speaker', 'published_date', 'comments', 'related_talks', 'title', 'film_date', 'tags', 'descript

In [40]:
#pickle files


Hi
